## Importing libraries & Reading files

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
import matplotlib.pyplot as plt
import datetime
from datetime import date
from tqdm.notebook import tqdm
print("Libraries Imported.")

# path of fundamental data file
f_path     = r"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\CSV\All Fundamental Data.csv"

# path of closing prices file 
c_path     = r"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\CSV\All Close Prices.csv"

# path of daily closing prices of Indexes : Nifty50 / NiftyMidcap100 / BSE500  as Sheetnames
index_path = r"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\CSV\Indexes Daily Closing.xlsx"


#reading files
fdf = pd.read_csv(f_path, encoding='windows-1252')
cdf = pd.read_csv(c_path, encoding='windows-1252')

print("file reading done.")
#fdf.drop(columns=['Unnamed: 0'],inplace=True)
cdf.drop(columns=['Unnamed: 0'],inplace=True)
cdf.drop(columns=['Unnamed: 0.1'],inplace=True)

Libraries Imported.
file reading done.


# Input Variables
Enter your Input varibales here, in the given format

In [60]:
start_year = 2009                   # Format : YYYY
                                    # Run 2004 onwards to get apt results.

buy_date   = '2023-07-01'           # Format : YYYY-MM-DD
sell_date  = '2024-07-01'           # Format : YYYY-MM-DD

indicator  = "ROCE"                 # Format : "ROCE" or "ROE"

m = 0                               # Top companies Ranking from m
n = 30                              # Top companies Ranking till n
                                    # Get top m to n companies;    here get Top 0-30 companies
    
p = 450                             # Get ranges till Rank 450th Company

transaction_cost = 0.1              # Percentage of Transaction cost eg.input 1 if you want 1% ; here 0.1 => 0.1%
capital = 1000000                   # Capital in the beginning of the backtest

weightage = "equal"                 # "equal" / "marketcap" / "rank"

a = 500                             # Market Cap greater than equal to a =   500Cr   |   Put a = 0 if no Greater than condition
b = 20000                           # Market Cap smaller than equal to b = 20000Cr   |   Put b = np.inf if no Less than condition

positionsizing = "hold_stocks"      # Position Sizing methods
                                    # "sell_all_stocks"  : Sell all stocks every year and buy again if repeated in next year
                                    # "hold_stocks"      : Hold the stocks that repeat in next year

### Getting data from inputs
No output for this code, just some minor calculations.

In [61]:
initial_capital   = capital

buy_date = pd.to_datetime(buy_date)
sell_date = pd.to_datetime(sell_date)

buy_str  = f"-{buy_date.month}-{buy_date.day}"
sell_str = f"-{sell_date.month}-{sell_date.day}"

r = n-m

# Getting End Year date

if buy_date.month > 3:
    end_year = buy_date.year - 2
else :
    end_year = buy_date.year - 3
    
print(end_year,r)

2021 30


----

## TRADE SHEET GENERATION CODE
Run this code to get the Trade Sheet
* **f1 :** *Reading Fundamental Data*
* **f2 :** *Store relevant columns of f1 after filters*
* **f3 :** *Top 30 (n) Stocks based on ROCE (indicator)*
* **f4 :** *Next Years Top 30 (n) Stocks to compare*
* **f5 :** *Previous Years Top 30 (n) Stocks to compare*
* **change :** *Adding every Year's Top 30 ROCE Stocks in this to get a complete list of all years*
* **TradeSheet :** *Final Tradesheet of all Years*

In [62]:
#------------------------------------------------  POSITION SIZING  -----------------------------------------------------
#                                                    CONDITION 1

combined_calc = pd.DataFrame(columns=["Top","CAGR","Trading Edge"])

for top in tqdm(range(0,int(p/n))):
    # print(f"{m} to {n} Started")
    capital = initial_capital


    if positionsizing == "sell_all_stocks":



        # Blank Dataframe to store Yearly Tradesheet
        change = pd.DataFrame()

        for year in tqdm(range(start_year, end_year+1)):

            f1 = fdf[fdf['Year']==year]

            # Fundamental Conditions
            conditions = ["Shareholder's Funds", "PAT", "PBIT"]

            for condition in conditions:
                f1 = f1[f1.fillna(0)[condition] > 0]

            # Market Cap Condition
            f1 = f1[(f1['Market Cap']>=a) & (f1['Market Cap']<=b)]

            cdf['Date'] = pd.to_datetime(cdf['Date'])
            cdf['Year'] = cdf['Date'].dt.year

            # Indicator Conditions
            if indicator == "ROCE" :

                # Remove Banking stocks in ROCE
                f1 = f1[~f1['Name'].str.contains('Bank')]

                # ROCE Formula
                f1[indicator] = (f1['Adj PBIT'].fillna(0) / (f1["Shareholder's Funds"].fillna(0) + f1["Total Debt"].fillna(0)))*100
                #f1[indicator] = (f1['Adj PBIT'].fillna(0) / (f1["Shareholder's Funds"].fillna(0) + f1["Long Term Borrowings"].fillna(0) + f1["Short Term Borrowings"].fillna(0)))*100

            elif indicator == "ROE" :

                # ROE Formula
                f1[indicator] = (f1['Adj PAT'].fillna(0) / f1["Shareholder's Funds"].fillna(0))*100


            # New dataframe to store calculations 
            f2 = f1[['Name','Market Cap',indicator]]                                                            # Name, Indicator
            f2['Buy Date'] = date(year+1,buy_date.month,buy_date.day)                                                        # Buy Date

            # Dataframe of companies containing closings on 1st July of start year & next year
            cnam  = cdf[cdf['Date'] == f'{year+1}{buy_str}']
            cnam2 = cdf[cdf['Date'] == f'{year+2}{sell_str}']


            # Reset Indexes
            f2.reset_index(drop=True,inplace=True)
            cnam.reset_index(drop=True,inplace=True)


            # Adding Closing Prices of Start year (Buying Price) and dropping companies with no Closing prices 
            f2 = pd.merge(f2,cnam[['Name','Close']], how='left',on='Name').dropna()
            f2.rename(columns={'Close':'Close1'},inplace=True)

            # SORTING VALUES BY INDICATOR
            f2 = f2.sort_values(by=[indicator], ascending=False)

            f2.reset_index(drop=True,inplace=True)                                                 # Reset Indexes

            # Adding Selling Date
            f2['Sell Date'] = date(year+2,sell_date.month,sell_date.day)

            # Adding Closing Prices of Next year (Selling Price) and dropping companies with no Closing prices 
            f2 = pd.merge(f2,cnam2[['Name','Close']], how='left',on='Name')
            f2.rename(columns={'Close':'Close2'},inplace=True)                                     # Reset Indexes

            # New Dataframe to store Top n Companies and Find Quantity
            f3 = f2[m:n]

            #display(f3)

            # Filling companies with no Selling Prices with last present close and update the Selling Dates
            for i in range(f3.shape[0]):
                if f3.fillna(0)['Close2'].iloc[i] == 0 :
                    f3['Close2'].iloc[i]    = cdf[(cdf['Name']==f3['Name'].iloc[i]) & (cdf['Date'] < f'{year+2}{sell_str}')].sort_values(by="Date", ascending=False)['Close'].iloc[0]
                    f3['Sell Date'].iloc[i] = cdf[(cdf['Name']==f3['Name'].iloc[i]) & (cdf['Date'] < f'{year+2}{sell_str}')].sort_values(by="Date", ascending=False)['Date'].iloc[0].date()

            #display(f3)

            # Claculating Quantity
            if weightage == "equal":
                f3['Qty'] = capital/((n-m)*f3['Close1'])
            elif weightage == "marketcap":
                f3['Qty'] = (capital * (f3['Market Cap']/f3['Market Cap'].sum())) / f3['Close1']
            elif weightage == "rank":
                new_weightage = f3.index[-1] - f3.index +1
                f3['Qty'] = (capital * (new_weightage/sum(f3.index))) / f3['Close1']



            # Defining a function to Roundown Quantity
            def Floor(x):
                try:
                    return math.floor(x)
                except:
                    return(x)




            # Rounddown Quantity
            f3['Qty'] = f3['Qty'].apply(Floor)


            # Qty Holding will be used to store the Quantity that we are holding since it was bought.
            f3['Qty Holding'] = f3['Qty']

            # Holding Amt will be used to store the Amount of Stock that we are holding since it was bought.
            f3['Holding Amt'] = f3['Qty']*f3['Close1']


            # Buying & Selling Value = Qty x Close Price
            f3['Buy Value'] = f3['Qty']*f3['Close1']
            f3['Sell Value'] = f3['Qty']*f3['Close2']

            f3['% Change'] = (f3['Close2'] / f3['Close1'] - 1)*100

            f3['Transaction Cost'] = (transaction_cost / 100)*(f3['Buy Value']+f3['Sell Value'])

            #print(f"{year} Starting Capital is {capital}")

            capital = f3['Sell Value'].sum() - f3['Transaction Cost'].sum()

            # print(f"{year} Ending Capital is {capital}")
            # print(year," Shape : ",f3.shape[0])
            change = pd.concat((change,f3),axis=0)
            #change = change.append(f3)



        # Displaying our Trade Sheet
        display(change)

        # CAGR
        cagr        = ((capital/initial_capital)**(1/(buy_date.year-start_year))-1)*100
        print(cagr)



        # Unique path according to the input variables to store the Trade File, same will be used to store the Trade Reports later.
        path_to = fr"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\Outputs\P1-{start_year}-{str(end_year)[-2:]}-{indicator}-Backtest-{initial_capital}-{buy_date.date()}-{sell_date.date()}-{positionsizing}-{m}-{n}-{transaction_cost}-{a} {b}-{weightage}-Reports.xlsx"

        #exporting the Trade Sheet
        change.to_excel(path_to,sheet_name="Trade Report Sheet")







    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

    #------------------------------------------------  POSITION SIZING  -----------------------------------------------------
    #                                                    CONDITION 2  





    elif positionsizing == "hold_stocks":


        # Blank Dataframe to store Yearly Tradesheet
        change = pd.DataFrame()
        f5 = pd.DataFrame()

        for year in tqdm(range(start_year, end_year+1)):

            #print(f"{int(capital)} : Starting Capital for {year} ")

            f1 = fdf[fdf['Year']==year]

            # Fundamental Conditions
            conditions = ["Shareholder's Funds", "PAT", "PBIT"]

            for condition in conditions:
                f1 = f1[f1.fillna(0)[condition] > 0]

            # Market Cap Condition
            f1 = f1[(f1.fillna(0)['Market Cap']>=a) & (f1.fillna(0)['Market Cap']<=b)]

            cdf['Date'] = pd.to_datetime(cdf['Date'])
            cdf['Year'] = cdf['Date'].dt.year

            # Indicator Conditions
            if indicator == "ROCE" :

                # Remove Banking stocks in ROCE
                f1 = f1[~f1['Name'].str.contains('Bank')]

                # ROCE Formula
                f1[indicator] = (f1['Adj PBIT'].fillna(0) / (f1["Shareholder's Funds"].fillna(0) + f1["Total Debt"].fillna(0)))*100
                #f1[indicator] = (f1['Adj PBIT'].fillna(0) / (f1["Shareholder's Funds"].fillna(0) + f1["Long Term Borrowings"].fillna(0) + f1["Short Term Borrowings"].fillna(0)))*100

            elif indicator == "ROE" :

                # ROE Formula
                f1[indicator] = (f1['Adj PAT'].fillna(0) / f1["Shareholder's Funds"].fillna(0))*100


            # New dataframe to store calculations 
            f2 = f1[['Name','Market Cap',indicator]]                                                            # Name, Indicator
            f2['Buy Date'] = date(year+1,buy_date.month,buy_date.day)                                           # Buy Date



            # Dataframe of companies containing closings on 1st July of start year & next year
            cnam  = cdf[cdf['Date'] == f'{year+1}{buy_str}']
            cnam2 = cdf[cdf['Date'] == f'{year+2}{sell_str}']


            # Reset Indexes
            f2.reset_index(drop=True,inplace=True)
            cnam.reset_index(drop=True,inplace=True)

            #f2.to_excel(fr"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\Outputs\Temp\f2 {year} before Cl.xlsx")

            # Adding Closing Prices of Start year (Buying Price) and dropping companies with no Closing prices 
            f2 = pd.merge(f2,cnam[['Name','Close']], how='left',on='Name').dropna()

    #         f2 = pd.merge(f2,cnam[['Name','Close']], how='left',on='Name')
    #         f2.to_excel(fr"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\Outputs\Temp\f2 {year} After Cl.xlsx")

    #         f2 = f2.dropna()
    #         f2.to_excel(fr"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\Outputs\Temp\f2 {year} After drop.xlsx")

            f2.rename(columns={'Close':'Close1'},inplace=True)

            f2['Sell Date' ] = date(year+2,sell_date.month,sell_date.day)

            # Adding Closing Prices of Next year (Selling Price) and dropping companies with no Closing prices 
            f2 = pd.merge(f2,cnam2[['Name','Close']], how='left',on='Name')
            f2.rename(columns={'Close':'Close2'},inplace=True)                                     # Reset Indexes




            # SORTING VALUES BY INDICATOR
            f2 = f2.sort_values(by=[indicator], ascending=False)

            f2.reset_index(drop=True,inplace=True)                                                 # Reset Indexes

            # New Dataframe to store Top n Companies and Find Quantity
            f3 = f2[m:n]

            #display(f3)


            # Filling companies with no Selling Prices with last present close and update the Selling Dates
            for i in range(f3.shape[0]):
                if f3.fillna(0)['Close2'].iloc[i] == 0 :
                    f3['Close2'].iloc[i]    = cdf[(cdf['Name']==f3['Name'].iloc[i]) & (cdf['Date'] < f'{year+2}{sell_str}')].sort_values(by="Date", ascending=False)['Close'].iloc[0]
                    f3['Sell Date'].iloc[i] = cdf[(cdf['Name']==f3['Name'].iloc[i]) & (cdf['Date'] < f'{year+2}{sell_str}')].sort_values(by="Date", ascending=False)['Date'].iloc[0].date()





            # Defining a function for Roundown
            def Floor(x):
                try:
                    return math.floor(x)
                except:
                    return(x)



            # Calculations for first year
            if year == start_year:

                f3['Prev Year'] = 0

                # Claculating Quantity
                if weightage == "equal":
                    f3['Qty'] = capital/((f3.shape[0])*f3['Close1'])
                elif weightage == "marketcap":
                    f3['Qty'] = (capital * (f3['Market Cap']/f3['Market Cap'].sum())) / f3['Close1']
                elif weightage == "rank":
                    new_weightage = f3.index[-1] - f3.index +1
                    f3['Qty'] = (capital * (new_weightage/sum(f3.index))) / f3['Close1']

                f3['Qty Holding'] = f3['Qty']    



            # Calculations for next years
            else:

                # Creating a column to store if the company is present in previous year.
                f3['Prev Year'] = f3['Name'].isin(f5['Name']).astype(int)

                n_prev = f3[f3['Prev Year'] == 0]                                          #Stocks not present in previous years
                y_prev = f3[f3['Prev Year'] == 1]                                          #Stocks present in previous years

                # Claculating Quantity for companies not present in previous year
                if weightage == "equal":
                    n_prev['Qty'] = capital/((n_prev.shape[0])*n_prev['Close1'])
                elif weightage == "marketcap":
                    n_prev['Qty'] = (capital * (n_prev['Market Cap']/n_prev['Market Cap'].sum())) / n_prev['Close1']
                elif weightage == "rank":
                    new_weightage = a.index[-1] - a.index +1
                    n_prev['Qty'] = (capital * (new_weightage/sum(f3.index))) / n_prev['Close1']


                n_prev['Qty Holding'] = n_prev['Qty']

                # Claculating Quantity for companies present in previous year
                y_prev[['Buy Date','Qty','Qty Holding']] = np.nan

                for k in range(y_prev.shape[0]):
                    y_prev['Qty Holding'].iloc[k] = f5[f5['Name'] == y_prev['Name'].iloc[k]]['Qty Holding'].iloc[0]

                # Merging above 2 dataframes back to f3
                f3 = pd.concat((y_prev,n_prev),axis=0)
                #f3 = y_prev.append(n_prev)

                f3.sort_values(by=[indicator],ascending=False, inplace=True)

            f3['Qty'] = f3['Qty'].apply(Floor)
            f3['Qty Holding'] = f3['Qty Holding'].apply(Floor)
            f3['Buy Value'] = f3['Qty']*f3['Close1']








            #----------------------------------------------NEXT YEAR COMPANIES--------------------------------------------------


            # Finding Next years Companies to compare

            f4 = fdf[fdf['Year']==(year+1)]

            conditions = ["Shareholder's Funds", "PAT", "PBIT"]

            for condition in conditions:
                f4 = f4[f4.fillna(0)[condition] > 0]

            f4 = f4[(f4.fillna(0)['Market Cap']>=a) & (f4.fillna(0)['Market Cap']<=b)]

            cdf['Date'] = pd.to_datetime(cdf['Date'])
            cdf['Year'] = cdf['Date'].dt.year


            if indicator == "ROCE" :
                f4 = f4[~f4['Name'].str.contains('Bank')]
                f4[indicator] = (f4['Adj PBIT'].fillna(0) / (f4["Shareholder's Funds"].fillna(0) + f4["Total Debt"].fillna(0)))*100
            elif indicator == "ROE" :
                f4[indicator] = (f4['Adj PAT'].fillna(0) / f4["Shareholder's Funds"].fillna(0))*100

            f4 = f4[['Name','Market Cap',indicator]]                                                            # Name, Indicator
            f4['Buy Date'] = date(year+2,buy_date.month,buy_date.day)                                           # Buy Date

            # Dataframe of companies containing closings on 1st July of start year & next year
            cnam3  = cdf[cdf['Date'] == f'{year+2}{buy_str}']
            cnam4  = cdf[cdf['Date'] == f'{year+3}{sell_str}']

            # Reset Indexes
            f4.reset_index(drop=True,inplace=True)
            cnam3.reset_index(drop=True,inplace=True)

            # Adding Closing Prices of Start year (Buying Price) and dropping companies with no Closing prices 
            f4 = pd.merge(f4,cnam3[['Name','Close']], how='left',on='Name').dropna()
            f4.rename(columns={'Close':'Close1'},inplace=True)

            f4['Sell Date' ] = date(year+3,sell_date.month,sell_date.day)

            # Adding Closing Prices of Next year (Selling Price) and dropping companies with no Closing prices 
            f4 = pd.merge(f4,cnam4[['Name','Close']], how='left',on='Name')
            f4.rename(columns={'Close':'Close2'},inplace=True)                                     # Reset Indexes

            # SORTING VALUES BY INDICATOR
            f4 = f4.sort_values(by=[indicator], ascending=False)

            f4.reset_index(drop=True,inplace=True)                                                 # Reset Indexes

            # New Dataframe to store Top n Companies and Find Quantity
            f4 = f4[m:n]

        #------------------------------------------------------------------------------------------------------------------------




            # Creating a column to store if the company is present in next year.
            f3['Next Year'] = f3['Name'].isin(f4['Name'].tolist()).astype(int)
            n_next = f3[f3['Next Year']==0]
            y_next = f3[f3['Next Year']==1]

            # Calculating Sell Value and Sell Date differently for above 2 dataframes
            n_next['Sell Value'] = n_next['Qty Holding']*n_next['Close2']
            y_next[['Sell Date','Sell Value']] = np.nan


            f3 = pd.concat((y_next,n_next),axis=0)
            #f3 = y_next.append(n_next)

            f3.sort_values(by=[indicator],ascending=False, inplace=True)


            f3['Transaction Cost'] = (transaction_cost / 100)*(f3.fillna(0)['Buy Value']+f3.fillna(0)['Sell Value'])

            f3['% Change'] = (f3['Close2']/f3['Close1']-1)*100 

            f5 = f3.copy()

            portfolio_value = (capital - f3.fillna(0)['Buy Value'].sum()) + ((f3['Qty Holding']*f3['Close2']) - f3['Transaction Cost']).sum()


            # Changing Capital for next year to Ending capital of this year 
            capital = (capital - f3.fillna(0)['Buy Value'].sum()) + f3.fillna(0)['Sell Value'].sum() - f3['Transaction Cost'].sum()
            #print(int(capital),": Ending Capital")


            # Appending every year's Trade file to a Dataframe "change"
            change = pd.concat((change,f3),axis=0)
            #change = change.append(f3)


        #------------------------------------------GETTING TRADESHEET FROM ABOVE DATAFRAME-----------------------------------------




        change2 = change.copy()
        s = pd.DataFrame(columns=change2.columns)
        change2.reset_index(drop=True, inplace=True)
        comp_list = []

        for i in range(change2.shape[0]):
            if change2['Name'].iloc[i] not in comp_list:

                t = change2[change2['Name']==change2['Name'].iloc[i]]
                t = t[(t.fillna(0)['Buy Date']!=0) | (t.fillna(0)['Sell Date']!=0)]

                comp_list.append(change2['Name'].iloc[i])

                for g in range(t.shape[0]):

                    if t.fillna(0)['Buy Date'].iloc[g] !=0 and t.fillna(0)['Sell Date'].iloc[g] !=0 :
                        s = pd.concat((s,t[g:g+1]),axis=0)
                        #s = s.append(t.iloc[g])
                    elif t.fillna(0)['Buy Date'].iloc[g] ==0 and t.fillna(0)['Sell Date'].iloc[g] ==0 :
                        continue
                    elif t.fillna(0)['Buy Date'].iloc[g] !=0 and t.fillna(0)['Sell Date'].iloc[g] ==0 :
                        s = pd.concat((s,t[g:g+1]),axis=0)
                        #s = s.append(t.iloc[g])

                    elif t.fillna(0)['Buy Date'].iloc[g] ==0 and t.fillna(0)['Sell Date'].iloc[g] !=0:

                        s['Sell Date'].iloc[-1]  = t['Sell Date'].iloc[g]
                        s['Close2'].iloc[-1]     = t['Close2'].iloc[g]
                        s['Sell Value'].iloc[-1] = t['Sell Value'].iloc[g]
            else:
                continue


        s['% Change'] = (s['Close2']/s['Close1']-1)*100      

        s = s.sort_values(by=['Buy Date','ROCE'], ascending=[True,False])


        TradeSheet = s[['Name','Market Cap','ROCE','Buy Date','Close1','Sell Date','Close2','Qty','Buy Value','Sell Value','Qty Holding','% Change','Transaction Cost']]
        TradeSheet.reset_index(drop=True,inplace=True)    

        cagr        = ((portfolio_value/initial_capital)**(1/(buy_date.year-start_year))-1)*100


        #display(change)
        #display(TradeSheet)


        # Unique path according to the input variables to store the Trade File, same will be used to store the Trade Reports later.
        path_to = fr"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\Outputs\P2-{start_year}-{str(end_year)[-2:]}-{indicator}-Backtest-{initial_capital}-{buy_date.date()}-{sell_date.date()}-{positionsizing}-{m}-{n}-{transaction_cost}-{a} {b}-{weightage}-Reports.xlsx"


        # Exporting files
        #change.to_excel(path_to,sheet_name="Trade Report Sheet")

        

        # TOP n Winners & Losers

        winners = change[change['% Change']>0].sort_values(by="% Change", ascending=False)[['Name','Buy Date','% Change']]
        losers  = change[change['% Change']<0].sort_values(by="% Change", ascending=True )[['Name','Buy Date','% Change']]
        winners.reset_index(drop=True, inplace=True)
        losers.reset_index(drop=True, inplace=True)

        # Count of Winners & Losers
        winners_counts = change[change['% Change']>0].shape[0] / change.shape[0]
        losers_counts  = change[change['% Change']<0].shape[0] / change.shape[0]

        # Average Win/Loss percentage
        avg_win  = change[change['% Change']>0]['% Change'].mean()
        avg_loss = change[change['% Change']<0]['% Change'].mean()

        # Ratios

        tradingedge = winners_counts*avg_win + losers_counts*avg_loss

        cagr        = ((portfolio_value/initial_capital)**(1/(buy_date.year-start_year))-1)*100


        # Dataframe to store the above Ratios
        calc = pd.DataFrame({"Top":[f"{m}-{n}"],"CAGR":[cagr],  "Trading Edge":[tradingedge]})
        
        combined_calc = pd.concat((combined_calc,calc),axis=0)
        #display(calc)



#         with pd.ExcelWriter(path_to,engine='openpyxl', mode='a',if_sheet_exists='replace') as writer : int +1
#             TradeSheet.to_excel(writer,sheet_name="Tradesheet",index=False)
#             calc.to_excel(writer,sheet_name="Calculations",index=False)
         
    
    m+=r
    n+=r
    
    combined_calc.to_excel(fr"C:\Users\admin\Dropbox\Purnanand\PMS\Fundamental Analysis\Main Files\Outputs\P2-{int(start_year+1)}-{end_year}--{m}-{n}Combined Calculations.xlsx",index=False)


    display(combined_calc)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

,Top,CAGR,Trading Edge
0,0-30,12.573558,18.448426


  0%|          | 0/13 [00:00<?, ?it/s]

,Top,CAGR,Trading Edge
0,0-30,12.573558,18.448426
0,30-60,16.428219,20.648145


...

,Top,CAGR,Trading Edge
0,0-30,12.573558,18.448426
0,30-60,16.428219,20.648145
0,60-90,13.254037,18.241525
0,90-120,11.633600,15.912665
0,120-150,12.357183,17.801916
0,150-180,16.135188,21.603076
0,180-210,13.819537,19.291755
0,210-240,11.118827,16.712359
0,240-270,8.582713,13.909449
0,270-300,9.834610,16.573186


  0%|          | 0/13 [00:00<?, ?it/s]

,Top,CAGR,Trading Edge
0,0-30,12.573558,18.448426
0,30-60,16.428219,20.648145
0,60-90,13.254037,18.241525
0,90-120,11.633600,15.912665
0,120-150,12.357183,17.801916
0,150-180,16.135188,21.603076
0,180-210,13.819537,19.291755
0,210-240,11.118827,16.712359
0,240-270,8.582713,13.909449
0,270-300,9.834610,16.573186


  0%|          | 0/13 [00:00<?, ?it/s]

,Top,CAGR,Trading Edge
0,0-30,12.573558,18.448426
0,30-60,16.428219,20.648145
0,60-90,13.254037,18.241525
0,90-120,11.633600,15.912665
0,120-150,12.357183,17.801916
0,150-180,16.135188,21.603076
0,180-210,13.819537,19.291755
0,210-240,11.118827,16.712359
0,240-270,8.582713,13.909449
0,270-300,9.834610,16.573186


----------